# Pre-trained models for text-to-speech

## SpeechT5

In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

In [ ]:
inputs = processor(
    text="Don't count the days, make the days count.",
    return_tensors="pt"
)

In [ ]:
from datasets import load_dataset

embeddings_dataset = load_dataset(
    "Matthijs/cmu-arctic-xvectors",
    split="validation"
)

import torch

speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

In [ ]:
spectrogram = model.generate_speech(
    inputs["input_ids"],
    speaker_embeddings=speaker_embeddings
)

In [ ]:
from transformers import SpeechT5HifiGan

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

In [ ]:
speech = model.generate_speech(
    inputs["input_ids"],
    speaker_embeddings,
    vocoder=vocoder
)

In [ ]:
from IPython.display import Audio

Audio(speech, rate=16000)

## Bark

In [ ]:
from transformers import BarkModel, BarkProcessor

model = BarkModel.from_pretrained("suno/bark-small")
processor = BarkProcessor.from_pretrained("suno/bark-small")

In [ ]:
# Add a speaker embedding
inputs = processor(
    "This is a test!",
    voice_preset="v2/en_speaker_3"
)
speech_output = model.generate(**inputs).cpu().numpy()

In [ ]:
# Try it in French, let's also add a French speaker embedding
inputs = processor(
    "C'est un test!",
    voice_preset="v2/fr_speaker_1"
)
speech_output = model.generate(**inputs).cpu().numpy()

In [ ]:
inputs = processor(
    "[Clears throat] This is a test... and I just took a long pause.",
    voice_preset="v2/fr_speaker_1"
)
speech_output = model.generate(**inputs).cpu().numpy()

In [ ]:
inputs = processor(
    "♪ In the mighty jungle, I'm trying to generate barks."
)
speech_output = model.generate(**inputs).cpu().numpy()

In [ ]:
input_list = [
    "[clears throat] Hello uh ..., my dog is cute [laughter]",
    "Let's try generating speech, with Bark, a text-to-speech model",
    "♪ In the jungle, the mighty jungle, the lion barks tonight ♪",
]

# Also add a speaker embedding
inputs = processor(
    input_list,
    voice_preset="v2/en_speaker_3"
)

speech_output = model.generate(**inputs).cpu().numpy()

In [ ]:
from IPython.display import Audio

sampling_rate = model.generation_config.sample_rate
Audio(speech_output[0], rate=sampling_rate)

In [ ]:
Audio(speech_output[1], rate=sampling_rate)

In [ ]:
Audio(speech_output[2], rate=sampling_rate)

## Massive Multilingual Speech (MMS)

In [ ]:
from transformers import VitsModel, VitsTokenizer

model = VitsModel.from_pretrained("facebook/mms-tts-deu")
tokenizer = VitsTokenizer.from_pretrained("facebook/mms-tts-deu")

In [ ]:
text_example = (
    "Ich bin Schnappi das kleine Krokodil, komm aus Ägypten das liegt direkt am Nil."
)

In [ ]:
import torch

inputs = tokenizer(text_example, return_tensors="pt")
input_ids = inputs["input_ids"]

with torch.no_grad():
    outputs = model(input_ids)

speech = outputs["waveform"]

In [ ]:
from IPython.display import Audio

Audio(speech, rate=16000)